# Imports

In [1]:
import numpy as np
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
from sklearn.metrics import confusion_matrix

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from numpy.random import seed
seed(888)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

## Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

## Get Data

In [4]:
y_train_all = np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)
y_train_all.shape

(60000,)

In [5]:
y_test = np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)
y_test.shape

(10000,)

In [6]:
x_train_all = np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)
x_train_all.shape

(60000, 784)

In [7]:
x_test = np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)
x_test.shape

(10000, 784)

## Explore

In [8]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [9]:
y_train_all[0:5]

array([5, 0, 4, 1, 9])

## Data Preprocessing

In [10]:
# Re-scale (Machiene can easily learn in range 0-1 than 0-255)
x_train_all , x_test = x_train_all/255 ,x_test/255

### Convert target value to One-Hot Encoding

In [11]:
# Array elemnet indexing using np.eye with return 1 as diagonal normally 
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [12]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [13]:
y_train_all.shape

(60000, 10)

In [14]:
y_test = np.eye(NR_CLASSES)[y_test]

In [15]:
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [16]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [17]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [18]:
x_train.shape

(50000, 784)

In [19]:
x_val.shape

(10000, 784)

## Setup Tensorflow Graph

In [20]:
X = tf.placeholder(tf.float32,shape=[None,TOTAL_INPUTS])
Y = tf.placeholder(tf.float32,shape=[None,NR_CLASSES])

### Hyperparameters

In [21]:
nr_epochs = 25
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 128
n_hidden3 = 32

In [22]:
#giving weights to the hidden layer 1
initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1],stddev=0.1,seed=42)
#creating a variable to hold weights because it has to hold new values again not perssitent to  initial values
w1 = tf.Variable(initial_value = initial_w1)

In [23]:
#giving bias to the hidden layer 1
initial_b1 = tf.constant(value=0.0,shape=[n_hidden1])
#creating a variable to hold bias because it has to hold new values again not perssitent to  initial values
b1 = tf.Variable(initial_value = initial_b1)

In [24]:
#cant see the values given to weights as tensorflow is two stage process.first setup only...
#initial_w1
#w1
#initial_b1
b1

<tf.Variable 'Variable_1:0' shape=(512,) dtype=float32_ref>

In [25]:
layer1_in = tf.matmul(X,w1) + b1

In [26]:
layer1_out = tf.nn.relu(layer1_in)

In [27]:
#FOR LAYER 2

initial_w2 = tf.truncated_normal(shape=[n_hidden1,n_hidden2],stddev=0.1,seed=42)
w2 = tf.Variable(initial_value = initial_w2)

initial_b2 = tf.constant(value=0.0,shape=[n_hidden2])
b2 = tf.Variable(initial_value = initial_b2)

layer2_in = tf.matmul(layer1_out,w2) + b2

layer2_out = tf.nn.relu(layer2_in)

In [28]:
#FOR LAYER 3

initial_w3 = tf.truncated_normal(shape=[n_hidden2,n_hidden3],stddev=0.1,seed=42)
w3 = tf.Variable(initial_value = initial_w3)

initial_b3 = tf.constant(value=0.0,shape=[n_hidden3])
b3 = tf.Variable(initial_value = initial_b3)

layer3_in = tf.matmul(layer2_out,w3) + b3

layer3_out = tf.nn.relu(layer3_in)

In [29]:
#FOR LAYER 4(OUT LAYER)

initial_w4 = tf.truncated_normal(shape=[n_hidden3,NR_CLASSES],stddev=0.1,seed=42)
w4 = tf.Variable(initial_value = initial_w4)

initial_b4 = tf.constant(value=0.0,shape=[NR_CLASSES])
b4 = tf.Variable(initial_value = initial_b4)

layer4_in = tf.matmul(layer3_out,w4) + b4

#As this is the last layer we need to use softmax rather than relu beacuse we need to get results in probability
output = tf.nn.softmax(layer4_in)

## Tensorboard Setup

In [30]:
# Folder for Tensorboard
timestr = time.strftime("%m %d - %H %M %S")
folder_name = "Model 1 at " +  timestr 
directory = os.path.join(LOGGING_PATH, folder_name)
try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


## Loss,Optimisation & Metrics

### Loss

In [31]:
# we will caluclaue the loss batch wise so we take reduced mean of them
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=output))

### Optimizer

In [32]:
#we are storing train_step so that we can run many times in loop can reduce loss
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

### Accuracy

In [33]:
#we cant apply the equal function on Y and output we need to find highest prob or value row-wise so argmax,axis is used
correct_predictions = tf.equal(tf.argmax(Y,axis = 1),tf.argmax(output,axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions,tf.float32))
print(accuracy)

Tensor("Mean_1:0", shape=(), dtype=float32)


In [34]:
TP = tf.count_nonzero(tf.argmax(output,axis = 1) * tf.argmax(Y,axis = 1))
TN = tf.count_nonzero((tf.argmax(output,axis = 1) - 1) * (tf.argmax(Y,axis = 1) - 1))
FP = tf.count_nonzero(tf.argmax(output,axis = 1) * (tf.argmax(Y,axis = 1) - 1))
FN = tf.count_nonzero((tf.argmax(output,axis = 1) - 1) * tf.argmax(Y,axis = 1))

In [35]:
tf.summary.scalar('accuracy',accuracy)
tf.summary.scalar('TP',TP)
tf.summary.scalar('TN',TN)
tf.summary.scalar('FP',FP)
tf.summary.scalar('FN',FN)

<tf.Tensor 'FN:0' shape=() dtype=string>

## Run Session

In [36]:
# all the above caluculations will work only inside session
sess = tf.Session()

### Setup FileWriter and merge Summaries

In [37]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '//train')
train_writer.add_graph(sess.graph)

### Initialise all the variables

In [38]:
init = tf.global_variables_initializer()
sess.run(init)

In [39]:
b4.eval(sess)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [40]:
w4.eval(sess)

array([[-3.18106147e-03, -1.11486413e-01, -9.49865803e-02,
         7.68418461e-02,  1.08649693e-01, -6.12520836e-02,
        -1.04762278e-01, -2.25129779e-02,  9.10906941e-02,
         1.03176966e-01],
       [-1.33157104e-01,  1.90708023e-02, -9.62553546e-02,
        -1.58192348e-02,  7.53587633e-02, -1.52259618e-01,
        -2.69646496e-02, -4.61504422e-02,  7.01877428e-03,
         1.06800616e-01],
       [-2.57340651e-02,  1.70168981e-01, -8.78938735e-02,
        -7.24191591e-02,  3.18428851e-03,  1.02812193e-01,
         1.23762123e-01, -1.06094144e-01, -1.06514748e-02,
         5.77867143e-02],
       [ 1.57123566e-01,  7.45584443e-02, -5.10990731e-02,
         1.21266425e-01, -1.10661365e-01, -1.39461622e-01,
        -1.79006085e-01, -4.23159339e-02, -1.59507412e-02,
        -1.07646873e-02],
       [-9.14043188e-02,  4.95505426e-03,  1.11035541e-01,
         1.42057642e-01,  1.46209702e-01,  1.08048610e-01,
        -1.90712079e-01,  6.38896748e-02,  1.56001672e-01,
        -2.

## Batching the Data

In [41]:
size_of_batch = 2000

In [42]:
num_examples = y_train.shape[0]

nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [43]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

## Training Loop

In [44]:
for epoch in range(nr_epochs):
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        feed_dictionary = {X:batch_x, Y:batch_y}
        sess.run(train_step, feed_dict=feed_dictionary)
    batch_accuracy = sess.run(fetches=[accuracy], feed_dict=feed_dictionary)
    print(batch_accuracy)
print('Done Training!')         

[0.742]
[0.7735]
[0.8755]
[0.8865]
[0.889]
[0.8925]
[0.894]
[0.894]
[0.896]
[0.8955]
[0.8955]
[0.898]
[0.898]
[0.8995]
[0.9005]
[0.901]
[0.902]
[0.9035]
[0.9825]
[0.986]
[0.987]
[0.9895]
[0.991]
[0.992]
[0.993]
Done Training!


In [45]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(test_accuracy)

0.9723


## Reset for the Next Run

In [46]:
train_writer.close()
sess.close()
tf.reset_default_graph()

In [47]:
# plt.figure(figsize=(7,7), dpi=95)
# plt.imshow(conf_matrix, cmap=plt.cm.Greens)

# plt.title('Confusion Matrix', fontsize=16)
# plt.ylabel('Actual Labels', fontsize=12)
# plt.xlabel('Predicted Labels', fontsize=12)

# tick_marks = np.arange(NR_CLASSES)
# plt.yticks(tick_marks, LABEL_NAMES)
# plt.xticks(tick_marks, LABEL_NAMES)

# plt.colorbar()

# for i, j in itertools.product(range(nr_rows), range(nr_cols)):
#     plt.text(j, i, conf_matrix[i, j], horizontalalignment='center',
#             color='white' if conf_matrix[i, j] > conf_matrix.max()/2 else 'black')
    

# plt.show()

NameError: name 'plt' is not defined

In [ ]:
# import seaborn as sn
# import pandas as pd
# import matplotlib.pyplot as plt
# array = [895, 9, 8, 0, 4, 6, 8, 3, 8, 5], 
# 	[2, 1007, 6, 1, 4, 2, 3, 9, 5, 9], 
# 	[1, 3, 918, 4, 8, 3, 5, 7, 1, 7], 
# 	[4, 3, 1, 1028, 3, 2, 7, 4, 2, 4], 
# 	[2, 3, 0, 8, 956, 3, 5, 1, 3, 2], 
# 	[0, 2, 3, 4, 6, 816, 2, 3, 2, 11], 
# 	[6, 1, 2, 3, 4, 7, 972, 2, 4, 2], 
# 	[0, 1, 2, 4, 7, 3, 5, 1014, 3, 10], 
# 	[6, 3, 2, 1, 3, 2, 3, 6, 945, 2], 
# 	[3, 1, 2, 3, 0, 2, 5, 0, 4, 1116]
# ]
# df_cm = pd.DataFrame(array, index = [i for i in "0123456789"],
#                   columns = [i for i in "0123456789"])
# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

In [ ]:
# import seaborn as sn